## image preprocessing

import the libs(some of them may not be used in this file though)

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline
import PIL

import numpy as np
import keras
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Flatten
from keras.models import Model
import keras.backend as k
from matplotlib import pyplot as plt
from IPython.display import clear_output

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import BoundaryAttack
from art.utils import to_categorical

Open the image

In [33]:
from PIL import Image, ImageOps, ImageEnhance
image = Image.open('demoImage/1.png')
#image.show()

Resize the image & Fit

It seems like similar :P

In [46]:
image.thumbnail((200, 200))
image.show()

result_image = ImageOps.fit(image, (50, 50))
result_image.show()

Generate noise + Add noise on the image

In [47]:
image = Image.open('demoImage/1.png').convert('RGBA')
iamge = ImageOps.fit(image,(256,256))
noise = Image.effect_noise(size=(256, 256), sigma=10)
noise = noise.convert('RGBA')

r,g,b,a = noise.split()
opacity = 0.4
alpha = ImageEnhance.Brightness(a).enhance(opacity)
noise.putalpha(alpha)
final = Image.new('RGBA',(256,256))
final = Image.alpha_composite(final,image)
final = Image.alpha_composite(final,noise)
final.show()

Grayscale

In [19]:
result_image = ImageOps.grayscale(image)
result_image.show()

Distortion

In [42]:
class WaveDeformer:
    def transform(self, x, y):
        y = y + 10*math.sin(x/40) 
        return x, y
    def transform_rectangle(self, x0, y0, x1, y1): 
        return (*self.transform(x0, y0),
                *self.transform(x0, y1),
                *self.transform(x1, y1),
                *self.transform(x1, y0),)                                                                   
    def getmesh(self, img): 
        self.w, self.h = img.size 
        gridspace = 20
        target_grid = []
        for x in range(0, self.w, gridspace):
            for y in range(0, self.h, gridspace): 
                target_grid.append((x, y, x + gridspace, y + gridspace))
    
        source_grid = [self.transform_rectangle(*rect) for rect in target_grid] 
        return [t for t in zip(target_grid, source_grid)]
        target_grid = []
        for x in range(0, self.w, gridspace):
            for y in range(0, self.h, gridspace): 
                target_grid.append((x, y, x + gridspace, y + gridspace))
                
        source_grid = [self.transform_rectangle(*rect) for rect in target_grid]

In [50]:
import math
image = Image.open('demoImage/dog.png')
result_image = ImageOps.deform(image, WaveDeformer())
result_image.save('results/distortedDog.png')